In [1]:
%matplotlib widget

import numpy as np
import ipympl
import matplotlib.pyplot as plt

In [4]:
from plot_france import plot_france

In [5]:
ax = plot_france()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [2]:
from plot_faster_dyn import plot_faster_dyn

In [3]:
plot_faster_dyn()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …